In [1]:
from pathling import PathlingContext, Expression as exp

pc = PathlingContext.create()

:: loading settings :: url = jar:file:/home/ivanc/.local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ivanc/.ivy2/cache
The jars for the packages stored in: /home/ivanc/.ivy2/jars
au.csiro.pathling#library-runtime added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3829cb50-3fc3-4e9d-9aed-840aab0793aa;1.0
	confs: [default]
	found au.csiro.pathling#library-runtime;7.2.0 in central
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 2401ms :: artifacts dl 24ms
	:: modules in use:
	au.csiro.pathling#library-runtime;7.2.0 from central in [default]
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta

In [2]:
data = pc.read.ndjson("data/fhir/cosri-demo")

In [3]:
result = data.extract("Patient",
                      columns=[
                          exp("name.first().given.first()", "Given name"),
                          exp("name.first().family", "Family name"),
                          exp("telecom.where(system = 'phone').value",
                              "Phone number"),
                      ],
                      )
display(result.first())


Row(Given name='Marcus', Family name='Aurelius', Phone number=None)

In [4]:
result.write.csv("patients_csv", header=True, mode="overwrite")

In [5]:
!cp patients_csv/part-*.csv patients.csv
!head patients.csv

Given name,Family name,Phone number
Marcus,Aurelius,
John,Cushing,
Harry,Osborn,
Elizabeth,Browning,
Charles,Dickens,


In [8]:
result = data.extract("QuestionnaireResponse",
                      filters=[
                          "questionnaire = 'Questionnaire/CIRG-PHQ9'"
                      ],                      columns=[
                          exp("questionnaire", "Questionnaire Reference"),
                          exp("item.where(linkId = '44261-6').answer.valueDecimal.first()", "PHQ-9 Score"),
                          exp("authored", "Date Authored"),
                          exp("subject.reference", "Patient Reference")
                      ])
display(result.first())


25/05/12 15:09:40 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


Row(Questionnaire Reference='Questionnaire/CIRG-PHQ9', PHQ-9 Score=Decimal('6.000000'), Date Authored='2021-12-24T20:02:11Z', Patient Reference='Patient/bfe05aa4-e21b-4ec6-ac19-8855f5af6552')